#  Hacker Earth problem 1

predict the selling price.

### Load Data

In [356]:
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objs as go

import plotly.io as pio
pio.renderers.default = "vscode"

from plotly import subplots

train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")

train_dataset.head(10)

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935
5,SHIECAYW3VZWGZGX,23.0,2015-09-25 16:31:46.000,162,Brilee,No,Technology,1,49,0.0,633.0,6.0,3959.0,6104.0,5631.247872
6,SHOE4FZGGTE7HJXN,33.0,2014-12-08 12:25:52.000,462,Zoe,No,Cosmetics,3,57,0.0,263.0,13.0,2006.0,5257.0,3874.264992
7,SHOE6QGAYFFMBWG6,4.0,2015-09-22 14:29:06.000,23,Koby,Yes,Educational,1,67,0.0,104.0,15.0,3596.0,5921.0,4820.943600
8,KRTEBSMKCGFESHYX,26.0,2016-03-28 21:48:01.000,156,Tiffani,Yes,Child_care,0,53,0.0,310.0,6.0,6965.0,10194.0,8837.724240
9,BRAE3P8Y8UVRY43K,5.0,2015-08-19 22:16:52.000,275,Nigel,Yes,Child_care,0,84,1.0,321.0,8.0,1103.0,5430.0,530.761862


In [357]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6368 entries, 0 to 6367
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_id        6368 non-null   object 
 1   Stall_no          6363 non-null   float64
 2   instock_date      6368 non-null   object 
 3   Market_Category   6368 non-null   int64  
 4   Customer_name     6157 non-null   object 
 5   Loyalty_customer  6368 non-null   object 
 6   Product_Category  6368 non-null   object 
 7   Grade             6368 non-null   int64  
 8   Demand            6368 non-null   int64  
 9   Discount_avail    6330 non-null   float64
 10  charges_1         6170 non-null   float64
 11  charges_2 (%)     6163 non-null   float64
 12  Minimum_price     6330 non-null   float64
 13  Maximum_price     6025 non-null   float64
 14  Selling_Price     6327 non-null   float64
dtypes: float64(7), int64(3), object(5)
memory usage: 746.4+ KB


### Drop missing

In [358]:
train_dataset.dropna(inplace=True)
test_dataset.dropna(inplace=True)
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5397 entries, 0 to 6367
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_id        5397 non-null   object 
 1   Stall_no          5397 non-null   float64
 2   instock_date      5397 non-null   object 
 3   Market_Category   5397 non-null   int64  
 4   Customer_name     5397 non-null   object 
 5   Loyalty_customer  5397 non-null   object 
 6   Product_Category  5397 non-null   object 
 7   Grade             5397 non-null   int64  
 8   Demand            5397 non-null   int64  
 9   Discount_avail    5397 non-null   float64
 10  charges_1         5397 non-null   float64
 11  charges_2 (%)     5397 non-null   float64
 12  Minimum_price     5397 non-null   float64
 13  Maximum_price     5397 non-null   float64
 14  Selling_Price     5397 non-null   float64
dtypes: float64(7), int64(3), object(5)
memory usage: 674.6+ KB


### Check for correlations

In [359]:
train_dataset[["Grade", "Demand", "Discount_avail", "charges_1", "charges_2 (%)", "Minimum_price", "Maximum_price", "Selling_Price"]].corr()

,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
Grade,1.000000,0.012805,-0.025084,-0.026668,-0.018252,-0.078824,0.166955,0.084998
Demand,0.012805,1.000000,0.000802,-0.006773,0.019977,0.007249,0.004238,0.002268
Discount_avail,-0.025084,0.000802,1.000000,-0.006346,0.022172,0.013266,-0.005766,-0.522162
charges_1,-0.026668,-0.006773,-0.006346,1.000000,0.273694,0.020651,0.008215,0.054814
charges_2 (%),-0.018252,0.019977,0.022172,0.273694,1.000000,0.008617,0.010466,0.008306
Minimum_price,-0.078824,0.007249,0.013266,0.020651,0.008617,1.000000,0.702970,0.689329
Maximum_price,0.166955,0.004238,-0.005766,0.008215,0.010466,0.702970,1.000000,0.761303
Selling_Price,0.084998,0.002268,-0.522162,0.054814,0.008306,0.689329,0.761303,1.000000


### Check for affect of product type

In [360]:
train_dataset[["Product_Category", "Minimum_price", "Maximum_price", "Selling_Price"]].groupby(by=["Product_Category"]).mean()

,Minimum_price,Maximum_price,Selling_Price
Product_Category,,,
Child_care,3550.920354,6506.315044,4531.994435
Cosmetics,3575.877005,6653.691622,4505.139299
Educational,3399.009488,6187.055028,4261.961944
Fashion,3625.200780,6611.288499,4885.348701
Home_decor,3577.374031,6582.587209,4608.748241
Hospitality,3561.122180,6552.050752,4507.265533
Organic,3416.057034,6451.249049,4331.926443
Pet_care,3614.292308,6471.365812,4545.796574
Repair,3548.432727,6567.321818,4628.220470


### Feature Selection

In [361]:
feature_columns = ["Minimum_price", "Maximum_price"]
label_column = ["Selling_Price"]

features = train_dataset[feature_columns]
label = train_dataset[label_column]

features.head()

,Minimum_price,Maximum_price
0,2983.0,4713.0
1,7495.0,10352.0
2,5752.0,7309.0
3,5090.0,20814.0
4,2430.0,9261.0


### Normalize data

In [362]:
features = features.to_numpy()
label = label.to_numpy()

def standardize(data: np.ndarray) -> np.ndarray:
    return (data - np.mean(data))/np.std(data)

def normalize(data: np.ndarray) -> np.ndarray:
    return ((data - np.min(data))/(np.max(data) - np.min(data)))

features = standardize(features)
label = standardize(label)

features = normalize(features)
label = normalize(label)


### Cluster data

In [363]:
cluster_dataset = np.concatenate([features, label], axis=1)
cluster_dataset.shape

(5397, 3)

In [364]:
max_values = 0.75
min_values = 0
step = 0.1

x_values = y_values = np.arange(min_values, max_values, step)

separation_boudary_1 = np.zeros((len(x_values), len(y_values)))
separation_boudary_2 = np.zeros((len(x_values), len(y_values)))
for x_index, x_val in enumerate(x_values):
    for y_index, y_val in enumerate(y_values):
        separation_boudary_1[x_index][y_index] = 0.3*x_val + 0.3*y_val + 0.217
        separation_boudary_2[x_index][y_index] = -0.1*x_val - 0.1*y_val + 0.15

In [365]:
clusters = np.zeros((len(label), 1))
for index in range(len(label)):
    if label[index] - (0.3*features[index, 0] + 0.3*features[index, 1] + 0.217) >= 0:
        clusters[index] = 1
    elif label[index] - (-0.1*features[index, 0] - 0.1*features[index, 1] + 0.15) <= 0:
        clusters[index] = 2
clusters.shape

(5397, 1)

In [366]:
clustered_dataset = np.concatenate([cluster_dataset, clusters], axis=1)
print(clustered_dataset.shape)

clustered_datasets = {}
for cluster in range(3):
    clustered_datasets[cluster] = {}
    clustered_datasets[cluster]["features"] = clustered_dataset[np.where(clustered_dataset[:, 3] == cluster), :2]
    clustered_datasets[cluster]["features"] = clustered_datasets[cluster]["features"].reshape((clustered_datasets[cluster]["features"].shape[1], 2))
    clustered_datasets[cluster]["label"] = clustered_dataset[np.where(clustered_dataset[:, 3] == cluster), 2].reshape((-1, 1))

(5397, 4)


### Apply regression on clusters

In [367]:
regression_planes = []
for cluster_index, cluster_values in clustered_datasets.items():
    regression = []
    x = tf.keras.Input(shape=(2,))
    y = tf.keras.layers.Dense(units=1, kernel_initializer="zeros")(x)
    model = tf.keras.Model(inputs=x, outputs=y)
    model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss=tf.keras.losses.MeanSquaredError())
    model.fit(cluster_values["features"], cluster_values["label"], epochs=30)
    regression.append(model.layers[1].get_weights()[0][0][0])
    regression.append(model.layers[1].get_weights()[0][1][0])
    regression.append(model.layers[1].get_weights()[1][0])
    regression_planes.append(regression)
print(regression_planes)

Epoch 1/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - loss: 0.0512
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - loss: 0.0370
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0267 
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - loss: 0.0181
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 816us/step - loss: 0.0114
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - loss: 0.0063
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - loss: 0.0029
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - loss: 8.5843e-04
Epoch 9/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - loss: 1.0892e-04
Epoch 10/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - loss: 1.5987e-05
Epoch 11/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 1.0579e-05
Epoch 12/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - loss: 8.7178e-06
Epoch 13/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.8968e-06 
Epoch 14/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - loss: 1.8686e-06
Epoch 15/30
26/26 ━━━━━━━━━━━━━

### Plot

In [377]:
plane_color = "#EFAC63"
cluster_colors = ["#907165", "#CBB578", "#DED8CD"]

price_3d_data = []
for cluster in range(3):
    price_3d_data.append(
        go.Scatter3d(
            x=clustered_datasets[cluster]["features"][:, 0],
            y=clustered_datasets[cluster]["features"][:, 1],
            z=clustered_datasets[cluster]["label"][:, 0],
            mode = "markers", name="Data Plot", showlegend=False,
            marker={"color": cluster_colors[cluster]}
        )
    )
price_3d_boundary1 = go.Surface(
    x=x_values, y=y_values, z=separation_boudary_1,
    name="Cluster Boundary 1", showscale=False, opacity=0.3,
    surfacecolor=np.ones_like(separation_boudary_1), colorscale=[[0, plane_color], [1, plane_color]]
)
price_3d_boundary2 = go.Surface(
    x=x_values, y=y_values, z=separation_boudary_2,
    name="Cluster boundary 2", showscale=False, opacity=0.3,
    surfacecolor=np.ones_like(separation_boudary_1), colorscale=[[0, plane_color], [1, plane_color]]
)
price_3d_regressions = []
for index, regression_values in enumerate(regression_planes):
    regression_plane = np.zeros((len(x_values), len(y_values)))
    for x_index, x_val in enumerate(x_values):
        for y_index, y_val in enumerate(y_values):
            regression_plane[x_index][y_index] = regression_values[0]*x_val + regression_values[1]*y_val + regression_values[2]
    price_3d_regressions.append(
        go.Surface(
            x=x_values, y=y_values, z=regression_plane,
            name=f"Regression Plane {index}", showscale=False, opacity=0.3,
            surfacecolor=np.ones_like(regression_plane), colorscale=[[0, plane_color], [1, plane_color]]
        )
    )

fig = subplots.make_subplots(
    rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'scene'}]], horizontal_spacing=0.1, subplot_titles=("Regression Plot",  "Clustering Plot")
    )
for data in price_3d_data:
    fig.add_trace(data, row=1, col=1)
for data in price_3d_data:
    fig.add_trace(data, row=1, col=2)
fig.add_trace(price_3d_boundary1, row=1, col=2)
fig.add_trace(price_3d_boundary2, row=1, col=2)
for trace in price_3d_regressions:
    fig.add_trace(trace, row=1, col=1)
fig.update_layout(
    template="plotly_dark", title="Hacker Earth Problem 1",
    scene1={"xaxis": {"title": "Maximum_price"}, "yaxis": {"title": "Minimum_price"}, "zaxis": {"title": "Selling_Price"}},
    scene2={"xaxis": {"title": "Maximum_price"}, "yaxis": {"title": "Minimum_price"}, "zaxis": {"title": "Selling_Price"}}
)
fig.write_html("price.html")

### Conclsion: Based on the minimum and maximum price of the product, one of the 3 linear models is chosen and applied to get the ideal selling price.